-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

# Advanced SQL Transformations

**Querying tabular data stored in the data lakehouse with Spark SQL is easy, efficient, and fast.**

This gets more complicated as the data structure becomes less regular, when many tables need to be used in a single query, or when the shape of data needs to be changed dramatically. This notebook introduces a number of functions present in Spark SQL to help engineers complete even the most complicated transformations.

## Learning Objectives
By the end of this lesson, you should be able to:
- Use **`.`** and **`:`** syntax to query nested data
- Work with JSON
- Flatten and unpacking arrays and structs
- Combine datasets using joins and set operators
- Reshape data using pivot tables
- Use higher order functions for working with arrays

## Run Setup

The setup script will create the data and declare necessary values for the rest of this notebook to execute.

In [0]:
%run ../Includes/Classroom-Setup-4.7

Python interpreter will be restarted.
Python interpreter will be restarted.

Creating the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_7"
Skipping install to "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss", dataset already exists

Cloning the sales table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/sales_hist...(7 seconds / 10,510 records)
Cloning the events table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/events_hist...(4 seconds / 485,696 records)
Cloning the events_update table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/events_update...(4 seconds / 1,927 records)
Cloning the events_raw table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/events_raw...(4 seconds / 2,252 records)
Cloning the item_lookup table from dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss/delta/item_lookup...(3 seconds / 12 records)

Predefined Paths:
 DA.paths.working_dir: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.7
 DA.paths.user_db: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.7/4_7.db
 DA.paths.datasets: dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/source/eltwss

Predefined tables in dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_7:
 events
 events_raw
 events_update
 item_lookup
 sales

Setup completed in 24 seconds

## Interacting with JSON Data

The **`events_raw`** table was registered against data representing a Kafka payload.

In most cases, Kafka data will be binary-encoded JSON values. We'll cast the **`key`** and **`value`** as strings below to look at these in a human-readable format.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW events_strings AS
  SELECT string(key), string(value) 
  FROM events_raw;
  
SELECT * FROM events_strings

key,value
UA000000107384208,"{""device"":""macOS"",""ecommerce"":{},""event_name"":""checkout"",""event_previous_timestamp"":1593880801027797,""event_timestamp"":1593880822506642,""geo"":{""city"":""Traverse City"",""state"":""MI""},""items"":[{""item_id"":""M_STAN_T"",""item_name"":""Standard Twin Mattress"",""item_revenue_in_usd"":595.0,""price_in_usd"":595.0,""quantity"":1}],""traffic_source"":""google"",""user_first_touch_timestamp"":1593879413256859,""user_id"":""UA000000107384208""}"
UA000000107388621,"{""device"":""Windows"",""ecommerce"":{},""event_name"":""email_coupon"",""event_previous_timestamp"":1593880770092554,""event_timestamp"":1593880829320848,""geo"":{""city"":""Hickory"",""state"":""NC""},""items"":[{""coupon"":""NEWBED10"",""item_id"":""M_STAN_F"",""item_name"":""Standard Full Mattress"",""item_revenue_in_usd"":850.5,""price_in_usd"":945.0,""quantity"":1}],""traffic_source"":""direct"",""user_first_touch_timestamp"":1593879889503719,""user_id"":""UA000000107388621""}"
UA000000107397512,"{""device"":""Windows"",""ecommerce"":{},""event_name"":""main"",""event_timestamp"":1593880824305898,""geo"":{""city"":""Fargo"",""state"":""ND""},""items"":[],""traffic_source"":""facebook"",""user_first_touch_timestamp"":1593880824305898,""user_id"":""UA000000107397512""}"
UA000000107369427,"{""device"":""Android"",""ecommerce"":{},""event_name"":""add_item"",""event_previous_timestamp"":1593880753875794,""event_timestamp"":1593880826675403,""geo"":{""city"":""Chicago"",""state"":""IL""},""items"":[{""item_id"":""M_STAN_T"",""item_name"":""Standard Twin Mattress"",""item_revenue_in_usd"":595.0,""price_in_usd"":595.0,""quantity"":1}],""traffic_source"":""facebook"",""user_first_touch_timestamp"":1593877772975990,""user_id"":""UA000000107369427""}"
UA000000107393216,"{""device"":""macOS"",""ecommerce"":{},""event_name"":""checkout"",""event_previous_timestamp"":1593880741311315,""event_timestamp"":1593880830140019,""geo"":{""city"":""Fargo"",""state"":""ND""},""items"":[{""item_id"":""M_STAN_F"",""item_name"":""Standard Full Mattress"",""item_revenue_in_usd"":945.0,""price_in_usd"":945.0,""quantity"":1}],""traffic_source"":""google"",""user_first_touch_timestamp"":1593880382734336,""user_id"":""UA000000107393216""}"
UA000000107325264,"{""device"":""Android"",""ecommerce"":{""purchase_revenue_in_usd"":1195.0,""total_item_quantity"":1,""unique_items"":1},""event_name"":""finalize"",""event_previous_timestamp"":1593876666372094,""event_timestamp"":1593880830038130,""geo"":{""city"":""Tuttle"",""state"":""OK""},""items"":[{""item_id"":""M_STAN_K"",""item_name"":""Standard King Mattress"",""item_revenue_in_usd"":1195.0,""price_in_usd"":1195.0,""quantity"":1}],""traffic_source"":""direct"",""user_first_touch_timestamp"":1593872552663730,""user_id"":""UA000000107325264""}"
UA000000107377602,"{""device"":""iOS"",""ecommerce"":{},""event_name"":""email_coupon"",""event_previous_timestamp"":1593880697501673,""event_timestamp"":1593880832779143,""geo"":{""city"":""Vineland"",""state"":""NJ""},""items"":[],""traffic_source"":""direct"",""user_first_touch_timestamp"":1593878686736308,""user_id"":""UA000000107377602""}"
UA000000107397643,"{""device"":""iOS"",""ecommerce"":{},""event_name"":""main"",""event_timestamp"":1593880838082657,""geo"":{""city"":""Minneapolis"",""state"":""MN""},""items"":[],""traffic_source"":""google"",""user_first_touch_timestamp"":1593880838082657,""user_id"":""UA000000107397643""}"
UA000000107369021,"{""device"":""iOS"",""ecommerce"":{},""event_name"":""reviews"",""event_previous_timestamp"":1593880064030570,""event_timestamp"":1593880834060045,""geo"":{""city"":""Spencer"",""state"":""WV""},""items"":[],""traffic_source"":""google"",""user_first_touch_timestamp"":1593877728194714,""user_id"":""UA000000107369021""}"
UA000000107388503,"{""device"":""Windows"",""ecommerce"":{},""event_name"":""delivery"",""event_previous_timestamp"":1593879876501779,""event_timestamp"":1593880838455915,""geo"":{""city"":""Hous

#### Spark SQL has built-in functionality to directly interact with JSON data stored as strings. We can use the **`:`** syntax to traverse nested data structures.

In [0]:
%sql
SELECT value:device, value:geo:city 
FROM events_strings

device,city
macOS,Traverse City
Windows,Hickory
Windows,Fargo
Android,Chicago
macOS,Fargo
Android,Tuttle
iOS,Vineland
iOS,Minneapolis
iOS,Spencer
Windows,Houston


#### Spark SQL also has the ability to parse JSON objects into struct types (a native Spark type with nested attributes).

However, the **`from_json`** function requires a schema. To derive the schema of our current data, we'll start by executing a query we know will return a JSON value with no null fields.

In [0]:
%sql
SELECT value 
FROM events_strings 
WHERE value:event_name = "finalize" 
ORDER BY key
LIMIT 3

value
"{""device"":""Linux"",""ecommerce"":{""purchase_revenue_in_usd"":1047.6,""total_item_quantity"":2,""unique_items"":2},""event_name"":""finalize"",""event_previous_timestamp"":1593879787820475,""event_timestamp"":1593879948830076,""geo"":{""city"":""Huntington Park"",""state"":""CA""},""items"":[{""coupon"":""NEWBED10"",""item_id"":""M_STAN_Q"",""item_name"":""Standard Queen Mattress"",""item_revenue_in_usd"":940.5,""price_in_usd"":1045.0,""quantity"":1},{""coupon"":""NEWBED10"",""item_id"":""P_DOWN_S"",""item_name"":""Standard Down Pillow"",""item_revenue_in_usd"":107.10000000000001,""price_in_usd"":119.0,""quantity"":1}],""traffic_source"":""email"",""user_first_touch_timestamp"":1593583891412316,""user_id"":""UA000000106459577""}"
"{""device"":""Windows"",""ecommerce"":{""purchase_revenue_in_usd"":1525.5,""total_item_quantity"":1,""unique_items"":1},""event_name"":""finalize"",""event_previous_timestamp"":1593880123694767,""event_timestamp"":1593880236431848,""geo"":{""city"":""Arlington"",""state"":""TX""},""items"":[{""coupon"":""NEWBED10"",""item_id"":""M_PREM_F"",""item_name"":""Premium Full Mattress"",""item_revenue_in_usd"":1525.5,""price_in_usd"":1695.0,""quantity"":1}],""traffic_source"":""email"",""user_first_touch_timestamp"":1593623684582377,""user_id"":""UA000000106581126""}"
"{""device"":""Android"",""ecommerce"":{""purchase_revenue_in_usd"":940.5,""total_item_quantity"":1,""unique_items"":1},""event_name"":""finalize"",""event_previous_timestamp"":1593879716269372,""event_timestamp"":1593879731743920,""geo"":{""city"":""Leitchfield"",""state"":""KY""},""items"":[{""coupon"":""NEWBED10"",""item_id"":""M_STAN_Q"",""item_name"":""Standard Queen Mattress"",""item_revenue_in_usd"":940.5,""price_in_usd"":1045.0,""quantity"":1}],""traffic_source"":""email"",""user_first_touch_timestamp"":1593632616907199,""user_id"":""UA000000106629972""}"


#### Spark SQL also has a **`schema_of_json`** function to derive the JSON schema from an example. Here, we copy and paste an example JSON to the function and chain it into the **`from_json`** function to cast our **`value`** field to a struct type.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW parsed_events AS
  SELECT from_json(value, schema_of_json('{"device":"Linux","ecommerce":{"purchase_revenue_in_usd":1075.5,"total_item_quantity":1,"unique_items":1},"event_name":"finalize","event_previous_timestamp":1593879231210816,"event_timestamp":1593879335779563,"geo":{"city":"Houston","state":"TX"},"items":[{"coupon":"NEWBED10","item_id":"M_STAN_K","item_name":"Standard King Mattress","item_revenue_in_usd":1075.5,"price_in_usd":1195.0,"quantity":1}],"traffic_source":"email","user_first_touch_timestamp":1593454417513109,"user_id":"UA000000106116176"}')) AS json 
  FROM events_strings;
  
SELECT * FROM parsed_events

json
"List(macOS, List(null, null, null), checkout, 1593880801027797, 1593880822506642, List(Traverse City, MI), List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1)), google, 1593879413256859, UA000000107384208)"
"List(Windows, List(null, null, null), email_coupon, 1593880770092554, 1593880829320848, List(Hickory, NC), List(List(NEWBED10, M_STAN_F, Standard Full Mattress, 850.5, 945.0, 1)), direct, 1593879889503719, UA000000107388621)"
"List(Windows, List(null, null, null), main, null, 1593880824305898, List(Fargo, ND), List(), facebook, 1593880824305898, UA000000107397512)"
"List(Android, List(null, null, null), add_item, 1593880753875794, 1593880826675403, List(Chicago, IL), List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1)), facebook, 1593877772975990, UA000000107369427)"
"List(macOS, List(null, null, null), checkout, 1593880741311315, 1593880830140019, List(Fargo, ND), List(List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1)), google, 1593880382734336, UA000000107393216)"
"List(Android, List(1195.0, 1, 1), finalize, 1593876666372094, 1593880830038130, List(Tuttle, OK), List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1)), direct, 1593872552663730, UA000000107325264)"
"List(iOS, List(null, null, null), email_coupon, 1593880697501673, 1593880832779143, List(Vineland, NJ), List(), direct, 1593878686736308, UA000000107377602)"
"List(iOS, List(null, null, null), main, null, 1593880838082657, List(Minneapolis, MN), List(), google, 1593880838082657, UA000000107397643)"
"List(iOS, List(null, null, null), reviews, 1593880064030570, 1593880834060045, List(Spencer, WV), List(), google, 1593877728194714, UA000000107369021)"
"List(Windows, List(null, null, null), delivery, 1593879876501779, 1593880838455915, List(Houston, TX), List(), facebook, 1593879876501779, UA000000107388503)"


#### Once a JSON string is unpacked to a struct type, Spark supports **`*`** (star) unpacking to flatten fields into columns.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW new_events_final AS
  SELECT json.* 
  FROM parsed_events;
  
SELECT * FROM new_events_final

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id
macOS,"List(null, null, null)",checkout,1593880801027797,1593880822506642,"List(Traverse City, MI)","List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))",google,1593879413256859,UA000000107384208
Windows,"List(null, null, null)",email_coupon,1593880770092554,1593880829320848,"List(Hickory, NC)","List(List(NEWBED10, M_STAN_F, Standard Full Mattress, 850.5, 945.0, 1))",direct,1593879889503719,UA000000107388621
Windows,"List(null, null, null)",main,null,1593880824305898,"List(Fargo, ND)",List(),facebook,1593880824305898,UA000000107397512
Android,"List(null, null, null)",add_item,1593880753875794,1593880826675403,"List(Chicago, IL)","List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))",facebook,1593877772975990,UA000000107369427
macOS,"List(null, null, null)",checkout,1593880741311315,1593880830140019,"List(Fargo, ND)","List(List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1))",google,1593880382734336,UA000000107393216
Android,"List(1195.0, 1, 1)",finalize,1593876666372094,1593880830038130,"List(Tuttle, OK)","List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))",direct,1593872552663730,UA000000107325264
iOS,"List(null, null, null)",email_coupon,1593880697501673,1593880832779143,"List(Vineland, NJ)",List(),direct,1593878686736308,UA000000107377602
iOS,"List(null, null, null)",main,null,1593880838082657,"List(Minneapolis, MN)",List(),google,1593880838082657,UA000000107397643
iOS,"List(null, null, null)",reviews,1593880064030570,1593880834060045,"List(Spencer, WV)",List(),google,1593877728194714,UA000000107369021
Windows,"List(null, null, null)",delivery,1593879876501779,1593880838455915,"List(Houston, TX)",List(),facebook,1593879876501779,UA000000107388503


## Explore Data Structures

Spark SQL has robust syntax for working with complex and nested data types.

Start by looking at the fields in the **`events`** table.

In [0]:
%sql
DESCRIBE events

col_name,data_type,comment
device,string,
ecommerce,struct,
event_name,string,
event_previous_timestamp,bigint,
event_timestamp,bigint,
geo,struct,
items,array>,
traffic_source,string,
user_first_touch_timestamp,bigint,
user_id,string,


The **`ecommerce`** field is a struct that contains a double and 2 longs.

We can interact with the subfields in this field using standard **`.`** syntax similar to how we might traverse nested data in JSON.

In [0]:
%sql
SELECT ecommerce.purchase_revenue_in_usd 
FROM events
WHERE ecommerce.purchase_revenue_in_usd IS NOT NULL

purchase_revenue_in_usd
1795.0
1045.0
535.5
1095.0
940.5
1995.0
940.5
1045.0
1695.0
1045.0


## Explode Arrays
The **`items`** field in the **`events`** table is an array of structs.

Spark SQL has a number of functions specifically to deal with arrays.

The **`explode`** function lets us put each element in an array on its own row.

In [0]:
%sql
SELECT user_id, event_timestamp, event_name, explode(items) AS item 
FROM events

user_id,event_timestamp,event_name,item
UA000000106466918,1593595620017592,add_item,"List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1)"
UA000000106541016,1593617822519726,guest,"List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1)"
UA000000106537762,1593623248104571,finalize,"List(null, M_PREM_Q, Premium Queen Mattress, 1795.0, 1795.0, 1)"
UA000000106544798,1593619763160248,guest,"List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1)"
UA000000106527304,1593733998028702,cart,"List(NEWBED10, M_PREM_T, Premium Twin Mattress, 985.5, 1095.0, 1)"
UA000000106506669,1593817553389549,cart,"List(NEWBED10, M_STAN_T, Standard Twin Mattress, 535.5, 595.0, 1)"
UA000000106550039,1593624602547189,cc_info,"List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1)"
UA000000106546519,1593618423460068,cart,"List(null, M_PREM_Q, Premium Queen Mattress, 1795.0, 1795.0, 1)"
UA000000106550923,1593621069581377,cart,"List(null, M_STAN_F, Standard Full Mattress, 945.0, 945.0, 1)"
UA000000106550923,1593621069581377,cart,"List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1)"


## Collect Arrays

The **`collect_set`** function can collect unique values for a field, including fields within arrays.

The **`flatten`** function allows multiple arrays to be combined into a single array.

The **`array_distinct`** function removes duplicate elements from an array.

Here, we combine these queries to create a simple table that shows the unique collection of actions and the items in a user's cart.

In [0]:
%sql
SELECT user_id,
  collect_set(event_name) AS event_history,
  array_distinct(flatten(collect_set(items.item_id))) AS cart_history
FROM events
GROUP BY user_id

user_id,event_history,cart_history
UA000000102368951,"List(checkout, cart, add_item, cc_info, finalize, register, shipping_info, mattresses)","List(M_PREM_K, M_STAN_F)"
UA000000102377152,"List(mattresses, cart, cc_info, finalize, register, shipping_info, checkout)",List(M_STAN_F)
UA000000102388362,List(finalize),"List(M_STAN_F, P_DOWN_S)"
UA000000102392998,List(mattresses),List()
UA000000102435405,"List(checkout, cart, add_item, cc_info, guest, finalize, shipping_info, mattresses)",List(M_STAN_T)
UA000000102462564,"List(mattresses, login, add_item, cart, cc_info, finalize, shipping_info, checkout)","List(M_STAN_T, P_FOAM_S, M_STAN_Q)"
UA000000102464478,"List(mattresses, cart, add_item, cc_info, guest, finalize, shipping_info, checkout)","List(M_PREM_T, P_FOAM_S)"
UA000000102477568,"List(mattresses, cart, add_item, cc_info, finalize, register, shipping_info, checkout)",List(M_STAN_Q)
UA000000102515405,"List(cc_info, finalize, register, shipping_info, checkout)",List(M_STAN_Q)
UA000000102516945,"List(mattresses, cart, add_item, cc_info, guest, finalize, shipping_info, checkout)",List(M_PREM_F)


## Join Tables

Spark SQL supports standard join operations (inner, outer, left, right, anti, cross, semi).

Here we chain a join with a lookup table to an **`explode`** operation to grab the standard printed item name.

In [0]:
%sql
CREATE OR REPLACE VIEW sales_enriched AS
SELECT *
FROM (
  SELECT *, explode(items) AS item 
  FROM sales) a
INNER JOIN item_lookup b
ON a.item.item_id = b.item_id;

SELECT * FROM sales_enriched

order_id,email,transaction_timestamp,total_item_quantity,purchase_revenue_in_usd,unique_items,items,item,item_id,name,price
285712,wbrown@gonzales-miranda.com,1592521889512254,2,1071.0,1,"List(List(NEWBED10, M_STAN_T, Standard Twin Mattress, 1071.0, 595.0, 2))","List(NEWBED10, M_STAN_T, Standard Twin Mattress, 1071.0, 595.0, 2)",M_STAN_T,Standard Twin Mattress,595.0
277921,campbellkatrina@phillips-duarte.com,1592458670364116,1,850.5,1,"List(List(NEWBED10, M_STAN_F, Standard Full Mattress, 850.5, 945.0, 1))","List(NEWBED10, M_STAN_F, Standard Full Mattress, 850.5, 945.0, 1)",M_STAN_F,Standard Full Mattress,945.0
274566,gregorytorres@meyer.com,1592419954844631,1,535.5,1,"List(List(NEWBED10, M_STAN_T, Standard Twin Mattress, 535.5, 595.0, 1))","List(NEWBED10, M_STAN_T, Standard Twin Mattress, 535.5, 595.0, 1)",M_STAN_T,Standard Twin Mattress,595.0
257606,ryanolson@brooks.com,1592213705353885,2,1754.0,2,"List(List(null, M_PREM_F, Premium Full Mattress, 1695.0, 1695.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1))","List(null, M_PREM_F, Premium Full Mattress, 1695.0, 1695.0, 1)",M_PREM_F,Premium Full Mattress,1695.0
257606,ryanolson@brooks.com,1592213705353885,2,1754.0,2,"List(List(null, M_PREM_F, Premium Full Mattress, 1695.0, 1695.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1))","List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1)",P_FOAM_S,Standard Foam Pillow,59.0
257628,kimberly68@mcpherson.net,1592214238807084,1,1045.0,1,"List(List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1))","List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1)",M_STAN_Q,Standard Queen Mattress,1045.0
289539,levans13@hotmail.com,1592572270401950,2,2016.0,2,"List(List(NEWBED10, M_STAN_K, Standard King Mattress, 1075.5, 1195.0, 1), List(NEWBED10, M_STAN_Q, Standard Queen Mattress, 940.5, 1045.0, 1))","List(NEWBED10, M_STAN_K, Standard King Mattress, 1075.5, 1195.0, 1)",M_STAN_K,Standard King Mattress,1195.0
289539,levans13@hotmail.com,1592572270401950,2,2016.0,2,"List(List(NEWBED10, M_STAN_K, Standard King Mattress, 1075.5, 1195.0, 1), List(NEWBED10, M_STAN_Q, Standard Queen Mattress, 940.5, 1045.0, 1))","List(NEWBED10, M_STAN_Q, Standard Queen Mattress, 940.5, 1045.0, 1)",M_STAN_Q,Standard Queen Mattress,1045.0
257794,anthonylopez@gmail.com,1592218840420069,1,1045.0,1,"List(List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1))","List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1)",M_STAN_Q,Standard Queen Mattress,1045.0
257918,wlarson@sanchez.info,1592221370370320,1,1695.0,1,"List(List(null, M_PREM_F, Premium Full Mattress, 1695.0, 1695.0, 1))","List(null, M_PREM_F, Premium Full Mattress, 1695.0, 1695.0, 1)",M_PREM_F,Premium Full Mattress,1695.0


## Set Operators
Spark SQL supports **`UNION`**, **`MINUS`**, and **`INTERSECT`** set operators.

**`UNION`** returns the collection of two queries. 

The query below returns the same results as if we inserted our **`new_events_final`** into the **`events`** table.

In [0]:
%sql
SELECT * FROM events 
UNION 
SELECT * FROM new_events_final

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id
Android,"List(null, null, null)",delivery,1593608889319762,1593608892119521,"List(La Habra, CA)",List(),google,1593607831910185,UA000000106497002
macOS,"List(null, null, null)",main,null,1593614356805317,"List(San Antonio, TX)",List(),facebook,1593614356805317,UA000000106526891
Chrome OS,"List(null, null, null)",warranty,1593601464446537,1593601767237755,"List(Salem, MA)",List(),instagram,1593601435427348,UA000000106477539
iOS,"List(null, null, null)",main,null,1593591162184972,"List(Flagstaff, AZ)",List(),youtube,1593591162184972,UA000000106462846
Android,"List(null, null, null)",main,null,1593618487291912,"List(Frisco, TX)",List(),google,1593618487291912,UA000000106549974
Windows,"List(null, null, null)",main,null,1593601400346620,"List(New York, NY)",List(),facebook,1593601400346620,UA000000106477460
macOS,"List(null, null, null)",original,1593597828746115,1593597836033567,"List(New York, NY)",List(),direct,1593597828746115,UA000000106470396
Android,"List(null, null, null)",add_item,1593614267924114,1593614767785568,"List(Bristol, CT)","List(List(null, M_STAN_T, Standard Twin Mattress, 595.0, 595.0, 1))",google,1593614267924114,UA000000106526430
Chrome OS,"List(null, null, null)",add_item,1593598140273117,1593598612510988,"List(Concord, NC)","List(List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1))",facebook,1593598140273117,UA000000106470943
Android,"List(null, null, null)",add_item,1593602059232593,1593602550671812,"List(New York, NY)","List(List(null, P_DOWN_S, Standard Down Pillow, 119.0, 119.0, 1))",instagram,1593602010363455,UA000000106478945


**`INTERSECT`** returns all rows found in both relations.

In [0]:
%sql
SELECT * FROM events 
INTERSECT 
SELECT * FROM new_events_final

device,ecommerce,event_name,event_previous_timestamp,event_timestamp,geo,items,traffic_source,user_first_touch_timestamp,user_id


The above query returns no results because our two datasets have no values in common.

**`MINUS`** returns all the rows found in one dataset but not the other; we'll skip executing this here as our previous query demonstrates we have no values in common.

## Pivot Tables
The **`PIVOT`** clause is used for data perspective. We can get the aggregated values based on specific column values, which will be turned to multiple columns used in **`SELECT`** clause. The **`PIVOT`** clause can be specified after the table name or subquery.

**`SELECT * FROM ()`**: The **`SELECT`** statement inside the parentheses is the input for this table.

**`PIVOT`**: The first argument in the clause is an aggregate function and the column to be aggregated. Then, we specify the pivot column in the **`FOR`** subclause. The **`IN`** operator contains the pivot column values. 

Here we use **`PIVOT`** to create a new **`transactions`** table that flattens out the information contained in the **`sales`** table.

This flattened data format can be useful for dashboarding, but also useful for applying machine learning algorithms for inference or prediction.

In [0]:
%sql
CREATE OR REPLACE TABLE transactions AS

SELECT * FROM (
  SELECT
    email,
    order_id,
    transaction_timestamp,
    total_item_quantity,
    purchase_revenue_in_usd,
    unique_items,
    item.item_id AS item_id,
    item.quantity AS quantity
  FROM sales_enriched
) PIVOT (
  sum(quantity) FOR item_id in (
    'P_FOAM_K',
    'M_STAN_Q',
    'P_FOAM_S',
    'M_PREM_Q',
    'M_STAN_F',
    'M_STAN_T',
    'M_PREM_K',
    'M_PREM_F',
    'M_STAN_K',
    'M_PREM_T',
    'P_DOWN_S',
    'P_DOWN_K'
  )
);

SELECT * FROM transactions

email,order_id,transaction_timestamp,total_item_quantity,purchase_revenue_in_usd,unique_items,P_FOAM_K,M_STAN_Q,P_FOAM_S,M_PREM_Q,M_STAN_F,M_STAN_T,M_PREM_K,M_PREM_F,M_STAN_K,M_PREM_T,P_DOWN_S,P_DOWN_K
ewalker@jordan-johnson.com,280515,1592493134477009,1,1195.0,1,null,null,null,null,null,null,null,null,1,null,null,null
james27@hotmail.com,413849,1593468897891582,1,59.0,1,null,null,1,null,null,null,null,null,null,null,null,null
cassie17@medina-anderson.com,445523,1593713036444518,2,2074.0,2,1,null,null,null,null,null,1,null,null,null,null,null
angela76@yahoo.com,291471,1592582917701512,1,945.0,1,null,null,null,null,1,null,null,null,null,null,null,null
gromero25@hotmail.com,458773,1593807106272431,2,2054.0,2,null,null,1,null,null,null,1,null,null,null,null,null
brayjose@carter.com,455266,1593791115255038,1,1195.0,1,null,null,null,null,null,null,null,null,1,null,null,null
wattsmarilyn@gmail.com,342102,1592934506123227,1,1195.0,1,null,null,null,null,null,null,null,null,1,null,null,null
tbrown21@gmail.com,288049,1592556531385950,1,107.10000000000001,1,null,null,null,null,null,null,null,null,null,null,1,null
cynthia7765@hotmail.com,419668,1593528333512589,1,1995.0,1,null,null,null,null,null,null,1,null,null,null,null,null
qpeterson@gmail.com,261103,1592246766873852,1,1195.0,1,null,null,null,null,null,null,null,null,1,null,null,null


## Higher Order Functions
Higher order functions in Spark SQL allow you to work directly with complex data types. When working with hierarchical data, records are frequently stored as array or map type objects. Higher-order functions allow you to transform data while preserving the original structure.

Higher order functions include:
- **`FILTER`** filters an array using the given lambda function.
- **`EXIST`** tests whether a statement is true for one or more elements in an array. 
- **`TRANSFORM`** uses the given lambda function to transform all elements in an array.
- **`REDUCE`** takes two lambda functions to reduce the elements of an array to a single value by merging the elements into a buffer, and the apply a finishing function on the final buffer.

## Filter
Remove items that are not king-sized from all records in our **`items`** column. We can use the **`FILTER`** function to create a new column that excludes that value from each array.

**`FILTER (items, i -> i.item_id LIKE "%K") AS king_items`**

In the statement above:
- **`FILTER`** : the name of the higher-order function <br>
- **`items`** : the name of our input array <br>
- **`i`** : the name of the iterator variable. You choose this name and then use it in the lambda function. It iterates over the array, cycling each value into the function one at a time.<br>
- **`->`** :  Indicates the start of a function <br>
- **`i.item_id LIKE "%K"`** : This is the function. Each value is checked to see if it ends with the capital letter K. If it is, it gets filtered into the new column, **`king_items`**

In [0]:
%sql
-- filter for sales of only king sized items
SELECT
  order_id,
  items,
  FILTER (items, i -> i.item_id LIKE "%K") AS king_items
FROM sales

order_id,items,king_items
285712,"List(List(NEWBED10, M_STAN_T, Standard Twin Mattress, 1071.0, 595.0, 2))",List()
277921,"List(List(NEWBED10, M_STAN_F, Standard Full Mattress, 850.5, 945.0, 1))",List()
274566,"List(List(NEWBED10, M_STAN_T, Standard Twin Mattress, 535.5, 595.0, 1))",List()
257606,"List(List(null, M_PREM_F, Premium Full Mattress, 1695.0, 1695.0, 1), List(null, P_FOAM_S, Standard Foam Pillow, 59.0, 59.0, 1))",List()
257628,"List(List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1))",List()
289539,"List(List(NEWBED10, M_STAN_K, Standard King Mattress, 1075.5, 1195.0, 1), List(NEWBED10, M_STAN_Q, Standard Queen Mattress, 940.5, 1045.0, 1))","List(List(NEWBED10, M_STAN_K, Standard King Mattress, 1075.5, 1195.0, 1))"
257794,"List(List(null, M_STAN_Q, Standard Queen Mattress, 1045.0, 1045.0, 1))",List()
257918,"List(List(null, M_PREM_F, Premium Full Mattress, 1695.0, 1695.0, 1))",List()
257798,"List(List(null, M_PREM_F, Premium Full Mattress, 1695.0, 1695.0, 1))",List()
258127,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))","List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))"


You may write a filter that produces a lot of empty arrays in the created column. When that happens, it can be useful to use a **`WHERE`** clause to show only non-empty array values in the returned column. 

In this example, we accomplish that by using a subquery (a query within a query). They are useful for performing an operation in multiple steps. In this case, we're using it to create the named column that we will use with a **`WHERE`** clause.

In [0]:
%sql
CREATE OR REPLACE TEMP VIEW king_size_sales AS

SELECT order_id, king_items
FROM (
  SELECT
    order_id,
    FILTER (items, i -> i.item_id LIKE "%K") AS king_items
  FROM sales)
WHERE size(king_items) > 0;
  
SELECT * FROM king_size_sales

order_id,king_items
289539,"List(List(NEWBED10, M_STAN_K, Standard King Mattress, 1075.5, 1195.0, 1))"
258127,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))"
258653,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))"
288311,"List(List(NEWBED10, P_FOAM_K, King Foam Pillow, 71.10000000000001, 79.0, 1))"
277614,"List(List(NEWBED10, M_PREM_K, Premium King Mattress, 1795.5, 1995.0, 1))"
259947,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))"
260236,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))"
260228,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))"
261148,"List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1))"
261103,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))"


## Transform
Built-in functions are designed to operate on a single, simple data type within a cell; they cannot process array values. **`TRANSFORM`** can be particularly useful when you want to apply an existing function to each element in an array. 

Compute the total revenue from king-sized items per order.

**`TRANSFORM(king_items, k -> CAST(k.item_revenue_in_usd * 100 AS INT)) AS item_revenues`**

In the statement above, for each value in the input array, we extract the item's revenue value, multiply it by 100, and cast the result to integer. Note that we're using the same kind as references as in the previous command, but we name the iterator with a new variable, **`k`**.

In [0]:
%sql
-- get total revenue from king items per order
CREATE OR REPLACE TEMP VIEW king_item_revenues AS

SELECT
  order_id,
  king_items,
  TRANSFORM (
    king_items,
    k -> CAST(k.item_revenue_in_usd * 100 AS INT)
  ) AS item_revenues
FROM king_size_sales;

SELECT * FROM king_item_revenues


order_id,king_items,item_revenues
289539,"List(List(NEWBED10, M_STAN_K, Standard King Mattress, 1075.5, 1195.0, 1))",List(107550)
258127,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))",List(119500)
258653,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))",List(119500)
288311,"List(List(NEWBED10, P_FOAM_K, King Foam Pillow, 71.10000000000001, 79.0, 1))",List(7110)
277614,"List(List(NEWBED10, M_PREM_K, Premium King Mattress, 1795.5, 1995.0, 1))",List(179550)
259947,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))",List(119500)
260236,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))",List(119500)
260228,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))",List(119500)
261148,"List(List(null, M_PREM_K, Premium King Mattress, 1995.0, 1995.0, 1))",List(199500)
261103,"List(List(null, M_STAN_K, Standard King Mattress, 1195.0, 1195.0, 1))",List(119500)


## Summary
Spark SQL offers a comprehensive set of native functionality for interacting with and manipulating highly nested data.

While some syntax for this functionality may be unfamiliar to SQL users, leveraging built-in functions like higher order functions can prevent SQL engineers from needing to rely on custom logic when dealing with highly complex data structures.

Run the following cell to delete the tables and files associated with this lesson.

In [0]:
%python
DA.cleanup()

Dropping the database "dbacademy_manujkumar_joshi_celebaltech_com_dewd_4_7"
Removing the working directory "dbfs:/user/manujkumar.joshi@celebaltech.com/dbacademy/dewd/4.7"

-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>